# Data Exploration

In [7]:
import pandas as pd
from pathlib import Path

# Local modules
from data_exploration import dataset_info

## Explore the dataset
- columns
- data format
- missing data (NaN, Null, etc.)

In [8]:
# Read the CSV into a DataFrame
csv = Path('Citibike_Data/JC-202307-citibike-tripdata.csv')
trip_df = pd.read_csv(csv)

In [9]:
# Get information about the dataset
print(f"The dataset contains {len(trip_df)} rows")
dataset_info(trip_df)

The dataset contains 106608 rows


,columns,dtypes,elements,missing,unique
0,ride_id,object,106608,0,106608
1,rideable_type,object,106608,0,3
2,started_at,object,106608,0,103107
3,ended_at,object,106608,0,103481
4,start_station_name,object,106601,7,103
5,start_station_id,object,106601,7,103
6,end_station_name,object,106216,392,191
7,end_station_id,object,106216,392,191
8,start_lat,float64,106608,0,34355
9,start_lng,float64,106608,0,35009


## Early observation
- 103 start startions but 191 end station, are some stations never used as starting points? Where are there?
- 392 end stations are missing, where the bikes stolen? Or just returned in the wild? See that only 122 end_lat/end_lng are missing, how far from a station are these bikes?
- More trips ended than started? Where they started the previous month?

## Follow up questions
- What dates are covered? Earliest/latest start and end dates?
- What are the ride types?
- How many members vs casual?

In [10]:
print(f"First start time: {min(trip_df['started_at'])}")
print(f"Last start time: {max(trip_df['started_at'])}")
print(f"First end time: {min(trip_df['ended_at'])}")
print(f"Last end time: {max(trip_df['ended_at'])}")

First start time: 2023-07-01 00:00:29
Last start time: 2023-07-31 23:59:44
First end time: 2023-07-01 00:03:56
Last end time: 2023-08-01 23:11:42


In [11]:
trip_df['rideable_type'].value_counts()

classic_bike     95055
electric_bike    11338
docked_bike        215
Name: rideable_type, dtype: int64

In [12]:
trip_df['member_casual'].value_counts()

member    73344
casual    33264
Name: member_casual, dtype: int64

## Most popular routes

In [23]:
start_stations = trip_df['start_station_id'].unique()
end_stations = trip_df['end_station_id'].unique()

print(f"There are {len(start_stations)} start stations.")
print(f"There are {len(end_stations)} end stations.")

There are 104 start stations.
There are 192 end stations.


In [60]:
station_matrix = []

for station in start_stations:
    connection_series = trip_df.loc[trip_df['start_station_id'] == station,'end_station_id'].value_counts()

    connection = {'start': station}

    for c in range(len(connection_series.index)):
        connection[connection_series.index[c]] = connection_series.values[c]

    station_matrix.append(connection)

connection_df = pd.DataFrame(station_matrix)
connection_df = connection_df.fillna(0)

In [61]:
connection_df

,start,JC109,JC103,JC077,JC115,JC078,JC020,JC098,JC066,JC002,...,7432.09,6022.04,5445.07,6197.08,7100.07,5593.04,5073.07,4953.04,5128.04,5288.09
0,JC077,123.0,32.0,27.0,26.0,21.0,13.0,12.0,12.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HB407,0.0,0.0,0.0,4.0,0.0,0.0,9.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC055,326.0,27.0,26.0,43.0,4.0,33.0,5.0,11.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC020,123.0,15.0,11.0,206.0,4.0,58.0,19.0,16.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC074,7.0,0.0,0.0,104.0,14.0,7.0,14.0,21.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,JC024,14.0,34.0,2.0,12.0,1.0,10.0,5.0,12.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,JC009,32.0,4.0,1.0,380.0,7.0,24.0,40.0,144.0,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
101,JC035,5.0,1.0,4.0,224.0,24.0,29.0,13.0,41.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
102,5351.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
